<a href="https://colab.research.google.com/github/visheshjain392/Movie-Recommendation/blob/main/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Pre-Processing

In [ ]:
# loading the data from the csv file to apandas datafram
# Deprecated in newer versions
movies_data = pd.read_csv('/content/movies.csv', on_bad_lines='skip')

In [ ]:
# print 5 first rows of the dataFrame
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800.0,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500.0,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.37678799999999,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466.0,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31295,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106.0,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124.0,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
# number of rows and columns in the data frame

movies_data.shape

(8463, 24)

In [ ]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [ ]:
# find the null values of the dataset
for feature in selected_features:
  print(feature, movies_data[feature].isnull().sum())

genres 56
keywords 807
tagline 1644
cast 88
director 62


In [ ]:
# replacing the null values with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [ ]:
# combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
8458    Action Crime Thriller united states\u2013mexic...
8459    Comedy Romance  A newlywed couple's honeymoon ...
8460    Comedy Drama Romance TV Movie date love at fir...
8461      A New Yorker in Shanghai Daniel Henney Eliza...
8462    Documentary obsession camcorder crush dream gi...
Length: 8463, dtype: object


In [ ]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 216638 stored elements and shape (8463, 17318)>
  Coords	Values
  (0, 201)	0.08092175327140651
  (0, 274)	0.09416699144055815
  (0, 5274)	0.114185824749548
  (0, 13599)	0.10501445554432536
  (0, 5437)	0.10501445554432536
  (0, 3678)	0.20815165566510663
  (0, 3065)	0.2182762878302597
  (0, 5836)	0.16431883053726912
  (0, 14378)	0.3439281299938501
  (0, 16587)	0.12452429856960913
  (0, 3225)	0.2574499583785319
  (0, 14271)	0.21337535944177008
  (0, 4945)	0.23541265891015098
  (0, 15261)	0.07047688519823454
  (0, 16998)	0.12882923209426486
  (0, 11192)	0.08857143427101599
  (0, 11503)	0.2843881862354024
  (0, 13349)	0.14871545320604207
  (0, 17007)	0.24031358729864064
  (0, 17290)	0.19759209055216423
  (0, 13319)	0.21573148180820126
  (0, 14064)	0.20284048800203613
  (0, 16668)	0.1949411304994076
  (0, 14608)	0.14769213857806932
  (0, 8756)	0.22407285580210667
  :	:
  (8461, 403)	0.17608623428966783
  (8461, 4835)	0.24334826303

In [ ]:
# getting the similarity scores using cosinne similarity

similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.07342448 0.03803202 ... 0.         0.         0.        ]
 [0.07342448 1.         0.0337127  ... 0.03446694 0.         0.        ]
 [0.03803202 0.0337127  1.         ... 0.         0.05267069 0.        ]
 ...
 [0.         0.03446694 0.         ... 1.         0.         0.02648969]
 [0.         0.         0.05267069 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02648969 0.         1.        ]]


In [ ]:
print(similarity.shape)

(8463, 8463)


In [ ]:
# getting the movie name from the user

movie_name = input("Enter the movie name: ")

Enter the movie name: X-man


In [ ]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
# finding the close match for the movie name given by the user
# Clean the list of titles to avoid float/NaN errors
list_of_all_titles = [str(title) for title in list_of_all_titles if pd.notna(title)]

# Make sure movie_name is a string too
movie_name = str(movie_name)

# Now find close matches
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['X-Men']


In [ ]:
close_match = find_close_match[0]
print(close_match)

X-Men


In [ ]:
# finding the index of the movie  with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

511


In [ ]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, np.float64(0.020989036278011953)), (1, np.float64(0.015301099858731213)), (2, np.float64(0.03479866772230368)), (3, np.float64(0.00946052824651859)), (4, np.float64(0.03362738628346501)), (5, np.float64(0.021866047938752537)), (6, np.float64(0.008887557000281735)), (7, np.float64(0.04796108059796191)), (8, np.float64(0.009015015832425104)), (9, np.float64(0.0742952410040264)), (10, np.float64(0.023537086394045614)), (11, np.float64(0.012925563690763196)), (12, np.float64(0.008935958296605775)), (13, np.float64(0.12447677063929656)), (14, np.float64(0.033807386217615434)), (15, np.float64(0.07211759569143045)), (16, np.float64(0.045067115628222104)), (17, np.float64(0.0086096423820813)), (18, np.float64(0.0)), (19, np.float64(0.007517913451981953)), (20, np.float64(0.02622396806940242)), (21, np.float64(0.0073371291677924635)), (22, np.float64(0.007848410403601817)), (23, np.float64(0.0075588689684701095)), (24, np.float64(0.011103882895414153)), (25, np.float64(0.03196761343733924

In [ ]:
len(similarity_score)

8463

In [ ]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(511, np.float64(1.0)), (3781, np.float64(1.0)), (3926, np.float64(0.18061611936477617)), (7190, np.float64(0.18061611936477617)), (1420, np.float64(0.15830545555017977)), (5485, np.float64(0.15830545555017977)), (294, np.float64(0.14352025264821924)), (2437, np.float64(0.13761363179307637)), (894, np.float64(0.13651689374127032)), (2870, np.float64(0.13538172936670284)), (3627, np.float64(0.13471363449829576)), (7090, np.float64(0.13471363449829576)), (1922, np.float64(0.1333592367685167)), (78, np.float64(0.1329696098645731)), (1231, np.float64(0.12973849453918274)), (13, np.float64(0.12447677063929656)), (899, np.float64(0.12414591133151535)), (45, np.float64(0.12056419373335667)), (1491, np.float64(0.11875209220892435)), (5556, np.float64(0.11875209220892435)), (3172, np.float64(0.11856103003947374)), (6833, np.float64(0.11856103003947374)), (76, np.float64(0.11778773640459207)), (657, np.float64(0.11701203769826139)), (1334, np.float64(0.11409174002239983)), (5399, np.float64(0.1

In [ ]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  if (i < 30):
    print(i, '.', title_from_index)
    i += 1

Movies suggested for you : 

1 . The Walk
2 . The Walk
3 . Don Jon
4 . Don Jon
5 . The Hundred-Foot Journey
6 . The Hundred-Foot Journey
7 . Inception
8 . 13 Hours: The Secret Soldiers of Benghazi
9 . Treasure Planet
10 . The Juror
11 . Shame
12 . Shame
13 . Sin City: A Dame to Kill For
14 . The Jungle Book
15 . Shutter Island
16 . The Lone Ranger
17 . In the Heart of the Sea
18 . World War Z
19 . The Conjuring
20 . The Conjuring
21 . Boiler Room
22 . Boiler Room
23 . G.I. Joe: The Rise of Cobra
24 . Ender's Game
25 . BloodRayne
26 . BloodRayne
27 . 10 Things I Hate About You
28 . 10 Things I Hate About You
29 . Sexy Beast


Movie Recommendation System

In [ ]:
movie_name = input("Enter the movie name: ")

# Clean the list of titles to avoid float/NaN errors
list_of_all_titles = [str(title) for title in movies_data['title'].tolist() if pd.notna(title)]

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

sorted_similar_movies = sorted(list(enumerate(similarity[index_of_the_movie])), key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  if (i < 30):
    print(i, '.', title_from_index)
    i += 1

Enter the movie name: Super man
Movies suggested for you : 

1 . Runner Runner
2 . Runner Runner
3 . Triple 9
4 . Triple 9
5 . Rounders
6 . Rounders
7 . The Duchess
8 . The Duchess
9 . Buffalo '66
10 . Buffalo '66
11 . Killing Them Softly
12 . Killing Them Softly
13 . Alex & Emma
14 . Alex & Emma
15 . Prince of Persia: The Sands of Time
16 . The Town
17 . The Town
18 . Owning Mahowny
19 . Owning Mahowny
20 . A Turtle's Tale: Sammy's Adventures
21 . A Turtle's Tale: Sammy's Adventures
22 . Argo
23 . The Lincoln Lawyer
24 . The Lincoln Lawyer
25 . The Gambler
26 . The Gambler
27 . The Infiltrator
28 . You Will Meet a Tall Dark Stranger
29 . You Will Meet a Tall Dark Stranger
